In [ ]:
### lrasmy@Zhilab last revised August 1 2022   ##
###############################################

## Download the required data 

- Make sure to mount your drive using the mount drive icon on the left pane or run the few lines below 



In [ ]:
### Mount your google drive ###
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


*   The data extracted will be in data/extracted_data




### Import the required packages


In [ ]:
import numpy as np
import pandas as pd
import sys
import os

sys.path.insert(0,"/content/drive/MyDrive/pytorch_ehr/Pytorch_EHR_Tutorial/Data_Prep/")
data_path='/content/drive/MyDrive/pytorch_ehr/Pytorch_EHR_Tutorial/Data_Prep/data/extracted_data/'


In [ ]:
pd.read_csv(data_path+'dhf_hard_label_v1.csv', sep='\t')

,patient,label,tte
0,0001054e-77c4-4e09-8310-7e5b1a579d8a,0,11907.0
1,0005ac2e-1174-4e89-8422-7783e5693d31,0,1934.0
2,000c35f5-35b4-4b7d-84f3-b022e01f7462,0,15483.0
3,000c4fd7-1fcf-4172-a59f-87553c0558a2,0,14315.0
4,001949a2-f330-42c0-84d1-a5de7bf0e992,0,5572.0
...,...,...,...
11735,ffe889aa-6020-4fb4-90ae-2ceb47042783,0,14469.0
11736,ffea6dfe-e087-4f7d-a19e-b795ceb7cbac,0,20727.0
11737,ffee4055-f5e4-49a5-958b-63a11e518025,0,19887.0
11738,fff8f5b5-417c-4b4a-bdd4-58e2f9cca750,1,2319.0


In [ ]:
pd.read_csv(data_path+'dhf_hard_data_v1_dmd.csv', sep='\t')

,patient,ecode,disc_dt
0,0001054e-77c4-4e09-8310-7e5b1a579d8a,D_15777000,1985-10-08
1,0001054e-77c4-4e09-8310-7e5b1a579d8a,D_162864005,1976-12-14
2,0001054e-77c4-4e09-8310-7e5b1a579d8a,D_44054006,1987-10-13
3,0001054e-77c4-4e09-8310-7e5b1a579d8a,a_31,1976-12-14
4,0001054e-77c4-4e09-8310-7e5b1a579d8a,a_40,1985-10-08
...,...,...,...
613637,fff9f7cc-90a2-422c-9ba2-8c209416ccd5,r_nonhispanic,1989-07-04
613638,fff9f7cc-90a2-422c-9ba2-8c209416ccd5,r_nonhispanic,1992-07-07
613639,fff9f7cc-90a2-422c-9ba2-8c209416ccd5,r_white,1988-02-23
613640,fff9f7cc-90a2-422c-9ba2-8c209416ccd5,r_white,1989-07-04


## Proprocess data into pickled list



In [ ]:
sys.path.insert(0,"/content/drive/MyDrive/pytorch_ehr/Pytorch_EHR_Tutorial/Data_Prep/")
from preprocess_outcomes  import dump_split_process_data

In [ ]:
output_proc='/content/drive/MyDrive/pytorch_ehr/Pytorch_EHR_Tutorial/Data_Prep/data/processed_data'
if not os.path.exists(output_proc):
        os.makedirs(output_proc)

dump_split_process_data(data_path+'dhf_easy_data_v1_dmd.csv', data_path+'dhf_easy_label_v1_e.csv' , 'NA' ,output_proc+'/dhf_easy_dmd_v1' , 'NA' )

loading data
loaded data for:  12119
loading labels
loaded labels for:  12119  after primary cleaning  12119
HF Cases counts:  4581
TTE > 1 year :  2688
new types dictionary
processed 1000 pts
processed 2000 pts
processed 3000 pts
processed 4000 pts
processed 5000 pts
processed 6000 pts
processed 7000 pts
processed 8000 pts
processed 9000 pts
processed 10000 pts
processed 11000 pts
processed 12000 pts
Reparsing
Splitting
random split


In [ ]:
dump_split_process_data(data_path+'dhf_hard_data_v1_dmd.csv', data_path+'dhf_hard_label_v1.csv' , output_proc+'/dhf_easy_dmd_v1.types' ,output_proc+'/dhf_hard_dmd_v1' , output_proc+'/dhf_easy_dmd_v1.pts' )

loading data
loaded data for:  11740
loading labels
loaded labels for:  11740  after primary cleaning  11740
HF Cases counts:  4405
TTE > 1 year :  11578
types dictionary loaded
processed 1000 pts
processed 2000 pts
processed 3000 pts
processed 4000 pts
processed 5000 pts
processed 6000 pts
processed 7000 pts
processed 8000 pts
processed 9000 pts
processed 10000 pts
processed 11000 pts
Reparsing
Splitting
loading previous splits
